In [1]:
!pip install openAI
!pip install Langchain
!pip install langchain[docarray]
!pip install tiktoken

In [44]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [45]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

QUESTION ONE 

Extract information about the columns of the Template table and tables A and B in the format of a text description.

In [46]:
TableA = 'table_A.csv'
loaderA = CSVLoader(file_path=TableA)
TableB = 'table_B.csv'
loaderB = CSVLoader(file_path=TableB)
Template = 'template.csv'
loader_Template = CSVLoader(file_path=Template)


In [47]:
index_A = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loaderA])

index_B = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loaderB])

index_template = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader_Template ])


In [51]:
query_A ="PLEASE Extract information about the columns of the TABLE_A  in the format of a text description."

In [52]:
response_A = index_A.query(query_A)

In [53]:
display(Markdown(response_A))

 The table contains information about four individuals who have purchased insurance plans. The columns include Date of Policy, Full Name, Insurance Plan, Policy Number, Monthly Premium, Department, Job Title, Policy Start, Insurance Type, and Monthly Cost.

In [54]:
query_B ="Similarly, please Extract information about the columns of the TABLE_B  in the format of a text description."
response_B = index_B.query(query_B)
display(Markdown(response_B))

 TABLE_B contains 8 columns: PolicyDate, Name, PlanType, Policy_ID, PremiumAmount, Hobby, MaritalStatus, StartDate, Employee_Name, Plan_Name, PolicyID, and Cost. The PolicyDate column contains the date the policy was created. The Name column contains the name of the policy holder. The PlanType column contains the type of plan the policy holder has. The Policy_ID column contains the unique identifier for the policy. The PremiumAmount column contains the amount of the premium for the policy. The Hobby column contains the policy holder's hobby. The MaritalStatus column contains the policy holder's marital status. The StartDate column contains the date the policy started. The Employee_Name column contains the name of the employee associated with the policy. The Plan_Name column contains the name of the plan associated with the policy. The PolicyID column contains the unique identifier for the policy. The Cost column contains the cost of the policy.

In [55]:
query_template ="Similarly, please Extract information about the columns of the Template  in the format of a text description."
response_template = index_template.query(query_template)
display(Markdown(response_template))

 The template contains four columns: Date, EmployeeName, Plan, PolicyNumber, and Premium. The Date column contains the date of the policy, the EmployeeName column contains the name of the employee, the Plan column contains the type of plan the employee has, the PolicyNumber column contains the policy number, and the Premium column contains the premium amount.

QUESTION 2 

For each of the candidate tables (A and B), ask the LLM to find similar columns in the Template.

In [56]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [57]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [58]:
docsA = loaderA.load()

In [59]:
dbA = DocArrayInMemorySearch.from_documents(
    docsA,
    embeddings
)

In [60]:
retriever_A = dbA.as_retriever()

In [61]:
docsB = loaderB.load()
dbB = DocArrayInMemorySearch.from_documents(
    docsB,
    embeddings
)
retriever_B = dbB.as_retriever()

In [62]:
docs_template= loader_Template.load()
dbtemp = DocArrayInMemorySearch.from_documents(
    docs_template,
    embeddings
)
retriever_template = dbtemp.as_retriever()

In [63]:
query = "Compare the columns in Table_A.csv with those in 'Template.csv' and identify which columns in Table A are the same as columns in Table T. For example 'DateOfPolicy' from Table_A is an alias to 'Date' from 'template' table "

In [69]:
query_A_template ="step 1.Compare the columns in Table A with those in template and identify which columns in Table A are having similar \
                   sentiments as columns in Table T. for example the 'Date_of_Policy' field from 'table_A' holds the same meaning  to the 'Date' field of table 'template'\
                   step 2. put the pairs of fields from the two tables in a tabular format "
response_A_template = index_template.query(query_A_template)
display(Markdown(response_A_template))

 The columns in Table A that have similar sentiments to the columns in Table T are EmployeeName, Plan, PolicyNumber, and Premium. The pairs of fields from the two tables in a tabular format are: 
Table A: EmployeeName, Plan, PolicyNumber, Premium 
Table T: Date, EmployeeName, Plan, PolicyNumber, Premium